# K-means clustering

## Overview

Clustering algorithms are a type of unsupervised machine learning algorithms that work by grouping datapoints based on their features. These algorithms are useful because they can be used to understand relationships in the data that aren't apparent when  inspected visually. There are a wide range of clustering algorithms that can be used, which include basic algorithms such as K-means to more complex and complicated algorithms such as DBSCAN or Agglomerative clustering. These differ in their assumptions regarding the number of cluster, shape of clusters or distance metrics.

In this project, we will use the K-means algorithm to try and cluster datapoints into groups using features from the iris dataset in `sklearn`. This will provide us with groups which we can compare to the actual classification to see how effective each algorithm is. 

## Import the required libraries

In [31]:
import pandas as pd
import numpy as np
from scipy import stats
from sklearn.cluster import KMeans
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.datasets import load_iris
from sklearn.metrics import adjusted_rand_score, normalized_mutual_info_score, fowlkes_mallows_score

## Importing and preparing the data

In this section, we import and scale our data, otherwise if a feature was not scaled and relatively larger compared to the other features, it may lead to poor performance by our model. This is due to how sensitive to the algorithm is to the scale of the features and by scaling, we ensure all features contribute equally to the clustering and there is no bias towards the larger features. It is important that after scaling our data that we then remove outliers, as the presence of outliers can significantly impact the algorithms calculation of the mean values for the centroids. This could result in incorrect or suboptimal centroids.

In [32]:
# Create instance of dataset
iris = load_iris()
X = iris.data
y = iris.target

In [34]:
# Scale the dataset
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Calculate the Z-score of each feature
z = np.abs(stats.zscore(X_scaled))

# Define a threshold for outliers
threshold = 3

# Find the indices of outliers
outlier_indices = np.where(z > threshold)

# Remove the outliers from the dataset
X_filtered = np.delete(X_scaled, outlier_indices[0], axis=0)
y_filtered = np.delete(y, outlier_indices[0], axis=0)

# Split the data into train and test set
x_train, x_test, y_train, y_test = train_test_split(X_filtered, y_filtered, test_size=0.25, random_state=42)

## Fitting the K-means algorithm

The K-means algorithm is a clustering technique that involves the following steps. Firstly, the number of clusters (k) is chosen, some common ways to do this include the Elbow or silhouette methods. Secondly, the k initial centroids are randomly chosen from the datapoints. Thirdly, each datapoint is assigned to the nearest cluster based on the k centroids. Fourthly, the centroid of each cluster is recalculated as the mean of all the datapoints in that cluster. The third and fourth steps are repeated until either the centroid positions no longer change significantly or the maximum number of iterations is reached. Finally, the algorithm returns the final set of k clusters.

The K-means algorithm is known for its simplicity and ability to handle larger datasets. It's simplicity means it can be adapted to handle different data types and use different distance metrics. However, it requires the number of clusters to be pre-determined and cannot handle when the clusters have non-globular shapes or have varying densities.

In [35]:
kmeans = KMeans(n_clusters=3, random_state=42, n_init=10)
kmeans.fit(x_train)
labels = kmeans.predict(x_test)

C:\Users\pjhop\anaconda3\lib\site-packages\sklearn\cluster\_kmeans.py:1382: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=1.
  warnings.warn(


## Evaluation of the clustering model

Since clustering is a unsupervised machine learning model and provides unlabelled clusters in its output, we cannot use the traditional machine learning metrics to evaluate our models. We instead split the metrics into internal and external validation methods to evaluate the clustering. Internal validation focuses on the quality of the cluster, in particular it focus on the structure of the data, intra-cluster and inter-cluster distances instead. External validation evaluates the quality of the cluster if the actual classes are available for comparison.

In our model, we use the following external validation metrics: Adjusted Rand Index (ARI), Normalized Mutual Information (NMI) and Fowlkes-Mallows Index (FMI). The ARI is used to determine whether two clusters are similar to one another, which is equal to one when the clusters are identical. The Normalized Mutual Information (NMI) is a measure of the mutual information between two sets of labels or clusters, adjusted for the size of the clusters. It ranges from 0 to 1, with 1 indicating perfect agreement between the two sets of labels. The Fowlkes-Mallows Index (FMI) is another external validation metric used to measure the similarity between two sets of labels or clusters. It is based on the geometric mean of the precision and recall of the clusters. Like the ARI and NMI, it ranges from 0 to 1, with 1 indicating perfect agreement between the two sets of labels.

In [36]:
adj_rand_score = adjusted_rand_score(y_test, labels)
nmi_score = normalized_mutual_info_score(y_test, labels)
fm_score = fowlkes_mallows_score(y_test, labels)
print(f"Adjusted Rand Score: {adj_rand_score:.3f}")
print(f"Normalized mutual info score: {nmi_score:.3f}")
print(f"Fowlkes Mallows score: {fm_score:.3f}")

Adjusted Rand Score: 0.670
Normalized mutual info score: 0.687
Fowlkes Mallows score: 0.777


Overall, based on these three evaluation metrics, it can be concluded that the clustering algorithm has produced a reasonably good clustering, but there is still room for improvement.